In [13]:
import cv2
import numpy as np
import shutil
import random
from pathlib import Path
from tqdm import tqdm



In [14]:
# ===============================
# CONFIGURATION
# ===============================
SOURCE_DIR = "/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/DataSets/raw"
TARGET_DIR = "/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/DataSets/processed/ThirdLab"

IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp'}
TARGET_COUNT = 300


In [15]:
def random_flip(img):
    return cv2.flip(img, 1)

def random_rotate(img):
    angle = random.uniform(-15, 15)
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    return cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REFLECT)

def random_noise(img):
    noise = np.random.normal(0, 10, img.shape).astype(np.int16)
    noisy = np.clip(img.astype(np.int16) + noise, 0, 255)
    return noisy.astype(np.uint8)

def random_erasing(img):
    h, w = img.shape[:2]
    erase_w = random.randint(w // 10, w // 4)
    erase_h = random.randint(h // 10, h // 4)
    x = random.randint(0, w - erase_w)
    y = random.randint(0, h - erase_h)
    img_copy = img.copy()
    img_copy[y:y + erase_h, x:x + erase_w] = random.randint(0, 255)
    return img_copy


In [16]:
# ===============================
# AUGMENTATION TIERS
# ===============================
LIGHT_AUG = [
    random_flip,
    random_rotate
]

MODERATE_AUG = [
    random_flip,
    random_rotate,
    random_noise
]

EXTENSIVE_AUG = [
    random_flip,
    random_rotate,
    random_noise,
    random_erasing
]


In [17]:
def augment_classes_to_300():
    src = Path(SOURCE_DIR)
    dst = Path(TARGET_DIR)
    dst.mkdir(parents=True, exist_ok=True)

    class_dirs = [d for d in src.iterdir() if d.is_dir()]

    total_ops = 0
    class_map = {}

    # Pre-calculate total operations
    for class_dir in class_dirs:
        images = [
            f for f in class_dir.iterdir()
            if f.is_file() and f.suffix.lower() in IMAGE_EXTENSIONS
        ]

        if len(images) < TARGET_COUNT:
            class_map[class_dir] = images
            total_ops += TARGET_COUNT

    # Process with progress bar
    with tqdm(total=total_ops, desc="Processing images") as pbar:
        for class_dir, images in class_map.items():

            num_images = len(images)

            # Select augmentation tier
            if num_images >= 200:
                AUGMENTATIONS = LIGHT_AUG
                tier = "LIGHT"
            elif num_images >= 100:
                AUGMENTATIONS = MODERATE_AUG
                tier = "MODERATE"
            else:
                AUGMENTATIONS = EXTENSIVE_AUG
                tier = "EXTENSIVE"

            print(f"\nClass: {class_dir.name} | Images: {num_images} | Augmentation: {tier}")

            target_class_dir = dst / class_dir.name
            target_class_dir.mkdir(exist_ok=True)

            # Copy original images
            for img_path in images:
                shutil.copy2(img_path, target_class_dir / img_path.name)
                pbar.update(1)

            current_count = num_images
            aug_index = 1

            # Generate augmented images
            while current_count < TARGET_COUNT:
                src_img_path = random.choice(images)
                img = cv2.imread(str(src_img_path))
                if img is None:
                    continue

                aug_func = random.choice(AUGMENTATIONS)
                aug_img = aug_func(img)

                out_name = f"{class_dir.name}_aug_{aug_index:04d}.jpg"
                cv2.imwrite(str(target_class_dir / out_name), aug_img)

                current_count += 1
                aug_index += 1
                pbar.update(1)

    print("\n✅ Augmentation completed successfully!")


In [ ]:
augment_classes_to_300()
